In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

In [3]:
league_id = 314
event = 33

In [4]:
def get_league_ids(league_id):
    mangers_standings = requests.get('https://fantasy.premierleague.com/api/leagues-classic/'+ str(league_id) +'/standings/').json()
    mangers = pd.DataFrame(mangers_standings['standings']['results'])
    mangers_ID = mangers['entry'].tolist()
    return mangers_ID

In [5]:
def get_players_element(mangers_ID):
    players_element = []
    for ID in mangers_ID:
        url = "https://fantasy.premierleague.com/api/entry/" + str(ID) + "/event/6/picks/"
        response = requests.get(url).json()
        mangers_picks = pd.DataFrame(response["picks"])["element"].tolist()
        for pick in mangers_picks:
            players_element.append(pick)
    return players_element

In [6]:
mangers_ID = get_league_ids(league_id)

In [7]:
data = get_players_element(mangers_ID) 

In [8]:
df = pd.DataFrame()
df['Players_ID'] = data

Players_ID = df['Players_ID'].value_counts().index.tolist()
Players_ID_times = df['Players_ID'].value_counts().tolist()

for rl in Players_ID_times:
    if rl < 20:
        rank = Players_ID_times.index(rl)
        break
        
Players_ID = Players_ID[:rank]
Players_ID_times = Players_ID_times[:rank]

In [9]:
df_of_players = pd.DataFrame()
df_of_players['ID'] = Players_ID
df_of_players['picks_num_in_league'] = Players_ID_times

In [10]:
players = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()
players_df = pd.DataFrame(players['elements'])
teams_df = pd.DataFrame(players['teams'])

teams=dict(zip(teams_df.id, teams_df.name))
players_df['team_name'] = players_df['team'].map(teams)

In [11]:
cols = [  'event_points', 
        'id', 'points_per_game', 'first_name', 'second_name', 'selected_by_percent', 'team_name' , 'form']

players = pd.DataFrame()

for col in cols:
    players[col]=players_df[col] 

In [12]:
players

,event_points,id,points_per_game,first_name,second_name,selected_by_percent,team_name,form
0,0,1,0.0,Folarin,Balogun,0.2,Arsenal,0.0
1,0,2,1.0,Cédric,Alves Soares,0.5,Arsenal,0.0
2,0,3,2.0,Mohamed,Elneny,0.1,Arsenal,0.0
3,0,4,2.3,Fábio,Ferreira Vieira,0.1,Arsenal,0.2
4,0,5,4.0,Gabriel,dos Santos Magalhães,28.3,Arsenal,4.5
...,...,...,...,...,...,...,...,...
846,0,788,0.0,Yerson,Mosquera,0.0,Wolves,0.0
847,0,800,0.0,Noha,Lemina,0.0,Wolves,0.0
848,0,832,0.0,Wesley,Okoduwa,0.0,Wolves,0.0
849,0,836,1.7,Leon,Chiwome,0.0,Wolves,1.2


In [13]:
datalist = []
Final = pd.DataFrame()

for ID in Players_ID:
    data = players[players['id'] == int(ID)]
    datalist.append(data)
    Final = pd.concat(datalist)

In [14]:
Final

,event_points,id,points_per_game,first_name,second_name,selected_by_percent,team_name,form
523,10,355,6.7,Erling,Haaland,70.7,Man City,5.0
17,2,19,6.4,Bukayo,Saka,58.3,Arsenal,3.8
137,3,108,5.0,Bryan,Mbeumo,6.8,Brentford,3.0
170,1,131,3.0,Pervis,Estupiñán,18.7,Brighton,0.8
632,0,28,2.5,Matt,Turner,9.6,Nott'm Forest,-0.2
591,0,405,3.0,Sven,Botman,4.6,Newcastle,0.0
545,15,373,4.5,Bruno,Borges Fernandes,11.3,Man Utd,8.5
512,5,343,4.5,Julián,Álvarez,16.4,Man City,2.2
616,0,430,4.3,Kieran,Trippier,22.9,Newcastle,0.0
438,2,308,7.2,Mohamed,Salah,39.3,Liverpool,4.5


In [15]:
Final['picks_num_in_league'] = Players_ID_times
result = Final.set_index("id")

In [16]:
result

,event_points,points_per_game,first_name,second_name,selected_by_percent,team_name,form,picks_num_in_league
id,,,,,,,,
355,10,6.7,Erling,Haaland,70.7,Man City,5.0,50
19,2,6.4,Bukayo,Saka,58.3,Arsenal,3.8,48
108,3,5.0,Bryan,Mbeumo,6.8,Brentford,3.0,41
131,1,3.0,Pervis,Estupiñán,18.7,Brighton,0.8,37
28,0,2.5,Matt,Turner,9.6,Nott'm Forest,-0.2,37
405,0,3.0,Sven,Botman,4.6,Newcastle,0.0,33
373,15,4.5,Bruno,Borges Fernandes,11.3,Man Utd,8.5,29
343,5,4.5,Julián,Álvarez,16.4,Man City,2.2,25
430,0,4.3,Kieran,Trippier,22.9,Newcastle,0.0,25


In [17]:
col = result.columns.tolist()
col

['event_points',
 'points_per_game',
 'first_name',
 'second_name',
 'selected_by_percent',
 'team_name',
 'form',
 'picks_num_in_league']

In [18]:
col2 = col[2:5] + col[7:8] + col[6:7] + col[1:2] + col[5:6]
result = Final[col2]
result

,first_name,second_name,selected_by_percent,picks_num_in_league,form,points_per_game,team_name
523,Erling,Haaland,70.7,50,5.0,6.7,Man City
17,Bukayo,Saka,58.3,48,3.8,6.4,Arsenal
137,Bryan,Mbeumo,6.8,41,3.0,5.0,Brentford
170,Pervis,Estupiñán,18.7,37,0.8,3.0,Brighton
632,Matt,Turner,9.6,37,-0.2,2.5,Nott'm Forest
591,Sven,Botman,4.6,33,0.0,3.0,Newcastle
545,Bruno,Borges Fernandes,11.3,29,8.5,4.5,Man Utd
512,Julián,Álvarez,16.4,25,2.2,4.5,Man City
616,Kieran,Trippier,22.9,25,0.0,4.3,Newcastle
438,Mohamed,Salah,39.3,23,4.5,7.2,Liverpool
